## Notebook : Vb-AGP + MCS 

In [2]:
import numpy as np
import scipy as sci
from scipy.stats import norm
import openturns as ot
from gaussian_process import GaussianProcess
from scipy import stats
#from AK_MCS_classic import AK_MCS




import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import rcParams
from matplotlib import rc

from matplotlib import cm

params = {'backend': 'ps',
      'axes.labelsize': 55,
      'legend.fontsize': 44,
      'xtick.labelsize': 40,
      'ytick.labelsize': 40,
      'text.usetex': True,
      'text.latex.unicode':True}

rcParams.update(params)

ModuleNotFoundError: No module named 'openturns'

# Cas test : Système en série à 4 branches

Illustration du cas test 

In [ ]:
def f4b_mesh(x,y):
    U1=3.+0.1*((x[:]-y[:])**2)-((x[:]+y[:])/np.sqrt(2))
    U2=3.+0.1*((x[:]-y[:])**2)+((x[:]+y[:])/np.sqrt(2))
     
    U3=x[:]-y[:]+(6/np.sqrt(2))
    U4=y[:]-x[:]+(6/np.sqrt(2))
#    U5=X[:,0]-X[:,1]+(7/np.sqrt(2))*np.ones(n).reshape(n,1)
#    U6=X[:,1]-X[:,0]+(7/np.sqrt(2))*np.ones(n).reshape(n,1)
    Z=np.minimum(U1,U2)
    Z=np.minimum(Z,np.array(U3))
    Z=np.minimum(Z,np.array(U4))
#    Z=np.minimum(Z,np.array(U5))
#    Z=np.minimum(Z,np.array(U6))
    return Z 

In [ ]:
delta=0.1
x = np.arange(-6.0,6.0, delta)
y = np.arange(-6.0, 6.0, delta)
Xm, Ym = np.meshgrid(x, y)
mshape=Xm.shape[0]
Xmf=Xm.flatten()
Ymf=Ym.flatten()
Z=f4b_mesh(Xmf,Ymf)    
Zm=Z.reshape(mshape,mshape)


fig = plt.figure(figsize=(14,8))
ax = fig.add_subplot(111)
ax.contourf(Xm, Ym, Zm,levels=[-10.,0.,100.],
             colors=('r','g'),alpha=0.05)
CS=ax.contour(Xm, Ym, Zm,levels=[0.],linestyles= 'dashed',
             colors='k')
CS.collections[0].set_label(r'$G(x)=0$')
ax.set_xlim([-6.,6.])
ax.set_ylim([-6.0,6.0]) 
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')

In [ ]:
def f_4b(X):
    U1=3+0.1*((X[0]-X[1])**2)-((X[0]+X[1])/np.sqrt(2))
    U2=3+0.1*((X[0]-X[1])**2)+((X[0]+X[1])/np.sqrt(2))
     
    U3=X[0]-X[1]+(6./np.sqrt(2))
    U4=X[1]-X[0]+(6./np.sqrt(2))
#    U5=X[:,0]-X[:,1]+(7/np.sqrt(2))*np.ones(n).reshape(n,1)
#    U6=X[:,1]-X[:,0]+(7/np.sqrt(2))*np.ones(n).reshape(n,1)
    Z=np.minimum(U1,U2)
    Z=np.minimum(Z,np.array(U3))
    Z=np.minimum(Z,np.array(U4))
#    Z=np.minimum(Z,np.array(U5))
#    Z=np.minimum(Z,np.array(U6))
    return Z 
stochastic_dim = 2
distribution_4b=ot.Normal(ot.Point([0]*stochastic_dim), ot.CovarianceMatrix(stochastic_dim))


Code algorithme Vb-AGP + MCS

In [ ]:

def Vb_AGP(performance_function,rv_distribution,initial_MC_sample_size = None,initial_doe_size = None, iter_max = 100, learning_function = 'U',cov_max = 0.1, hot_start = False,path='',corr='matern52'):
    var_G_E_X_l=list()
    var_X_E_G_l=list()
    var_tot_l=list()
    var_tot=0.
    var_tot_inf=0.
    var_tot_sup=0.

    
    if hot_start == False: # new initial DoE, new initial MC
        #initial MC population
        MC_sample_size = initial_MC_sample_size
        MC_sample = np.array(rv_distribution.getSample(MC_sample_size))
        # initial DoE generation
        lhs = ot.LHSExperiment(rv_distribution, initial_doe_size)
        lhs.setAlwaysShuffle(True) # randomized
        spaceFilling = ot.SpaceFillingC2()
        N = 50000
        optimalLHSAlgorithm = ot.MonteCarloLHS(lhs, N, spaceFilling)
        DoE=np.array(optimalLHSAlgorithm.generate())
        DoE_size = DoE.shape[0]
        # inital DoE evaluation 
        print("--------------------------------------")
        print("Evalulation of the perfomance function on the initial doe of size ",initial_doe_size)
        print("--------------------------------------")
       # Evaluation of the performance function
        Y = np.zeros((initial_doe_size,1))
        i = 0
        for x in DoE:
            Y[i,0] = performance_function(x)
            print(Y[i,0])
            i = i+1
        print( "--------------------------------------")
        print("Saving the initial DoE ")
        print( "--------------------------------------")
        MC_sample.dump(path+str('MC_sample_init.pk'))
        DoE.dump(path+str('DoE_init.pk'))
        Y.dump(path+str('Y_init.pk'))
        
    elif hot_start == 'init': # reuse of the previous initial MC population and inital DoE
        MC_sample = np.load(path+str('MC_sample_init.pk'),allow_pickle=True)
        MC_sample_size = MC_sample.shape[0] 
        initial_MC_sample_size =MC_sample_size 
        DoE = np.load(path+str('DoE_init.pk'),allow_pickle=True)
        DoE_size = DoE.shape[0]
        Y = np.load(path+str('Y_init.pk'),allow_pickle=True)
        print( "--------------------------------------")
        print ("Reading the initial DoE of size ",DoE_size)
        print( "--------------------------------------")
 

    elif hot_start == 'init_doe': # new MC populaton and reuse of the initial DoE
        MC_sample_size = initial_MC_sample_size
        MC_sample = np.array(rv_distribution.getSample(MC_sample_size))
        initial_MC_sample_size =MC_sample_size 
        DoE = np.load(path+str('DoE_init.pk'),allow_pickle=True)
        DoE_size = DoE.shape[0]
        Y = np.zeros((DoE_size,1))
        i = 0
        for x in DoE:
            Y[i,0] = performance_function(x)
            print(Y[i,0])
            i = i+1
        print( "--------------------------------------")
        print ("Reading the initial DoE of size ",DoE_size)
        print( "--------------------------------------")

        
        
    elif hot_start == True: # reuse of the MC population and DoE (last update)
        MC_sample = np.load(path+str('MC_sample.pk'),allow_pickle=True)
        MC_sample_size = MC_sample.shape[0]  
        initial_MC_sample_size =MC_sample_size 
        DoE = np.load(path+str('DoE.pk'),allow_pickle=True)
        DoE_size = DoE.shape[0]
        Y = np.load(path+str('Y.pk'),allow_pickle=True)        
        print( "--------------------------------------")
        print ("Hot start with a DoE of size ",DoE_size)
        print( "--------------------------------------")
        
    #construction of the GP surrogate
    stochastic_dim = DoE.shape[1]    
    theta0 = np.array([0.1]*stochastic_dim)
    thetaL = np.array([1e-6]*stochastic_dim)
    thetaU = np.array([100.0]*stochastic_dim)
    gp_g = GaussianProcess(corr=corr,theta0 = theta0,thetaL=thetaL,thetaU=thetaU)     
    #Vb-AGP loop
    n_iter = 0
    convergence = False

    while n_iter<iter_max and convergence == False:
        #fitting the GP surrogate
        gp_g.fit(DoE,Y)
        #estimation of the performance function surrogate model at the MC points
        MC_sample_G,MSE = gp_g.predict(MC_sample,eval_MSE=True)

        #estimation of the probability of failure Pf
        indicatrice = np.zeros((MC_sample_size,1))
        ind = MC_sample_G<=0.0
        indicatrice[ind] = 1.0
        Pf = indicatrice.sum()/MC_sample_size
        
        print( "--------------------------------------")
        print ("iteration number ",n_iter)
        print ("Pf = ",Pf)
        
        if Pf > 0. :    
            # using the function sensitivity_analysis : computation of variances var_G_E_X (GP) et var_X_E_G (MC) 
            # +  control of the stopping criterion 1st step (COV_red) : end_learning = boolean 
            var_X_E_G, var_G_E_X,end_learning,var_X_E_G_inf,var_X_E_G_sup,var_G_E_X_inf,var_G_E_X_sup=sensitivity_analysis(gp_g,rv_distribution,MC_sample,Pf,cov_max)
            var_G_E_X_l.append(var_G_E_X)
            var_X_E_G_l.append(var_X_E_G)
            np.array(var_X_E_G_l).dump('var_X_E_G.pk')   
            np.array(var_G_E_X_l).dump('var_G_E_X.pk')   

            print('cov_MC = ', np.sqrt(var_X_E_G)/Pf)
            print('cov_GP = ',np.sqrt(var_G_E_X)/Pf)


            # check of COV_tot if 1st stopping criterion verified (COV_red)
            if end_learning : 
                # computation of the total variance : var_tot
                print('Computation of the total COV')
                var_tot,var_tot_inf,var_tot_sup,Pf=total_variance_bootstrap(gp_g,rv_distribution,MC_sample_size,Pf,cov_max,MC_sample)
                var_tot_l.append(var_tot)
                np.array(var_tot_l).dump('var_tot.pk')
                var_target=(cov_max*Pf)**2 # limit value of the variance (corresponding to cov_max)
                print('cov_tot = ',np.sqrt(var_tot)/Pf)
                # check of the learning stopping criterion 
                if var_tot_sup < var_target :
                    convergence = True 
                else : 
                    end_learning = False
                    
            # if stopping criterion not completed                              
            if end_learning == False :
                test_var = var_X_E_G > var_G_E_X
                if test_var : # MC improvement
                    print('higher variance due to : MC')
                    print( "NEW MC SAMPLE")
                    #adding points to the MC population
                    new_MC_sample = np.zeros((MC_sample.shape[0]+initial_MC_sample_size,MC_sample.shape[1]))
                    new_MC_sample[0:MC_sample.shape[0],:] = MC_sample
                    new_MC_sample[MC_sample.shape[0]:,:] = np.array(rv_distribution.getSample(initial_MC_sample_size))
                    MC_sample = new_MC_sample.copy()
                    MC_sample_size = MC_sample_size + initial_MC_sample_size
                    
                    
                else : # GP improvment 
                    print('higher variance due to : GP')
                    # search of the best learning point x_star among the MC population using the EFF criterion 
                    epsilon = 2.0*np.sqrt(MSE)
                    X = MC_sample_G[:,0]/np.sqrt(MSE)
                    X1 = (-epsilon-MC_sample_G[:,0])/np.sqrt(MSE)
                    X2 = (epsilon-MC_sample_G[:,0])/np.sqrt(MSE)
                    EFF = MC_sample_G[:,0]*(2.0*sci.stats.norm.cdf(-X)-sci.stats.norm.cdf(X1)-sci.stats.norm.cdf(X2))-\
                    np.sqrt(MSE)*(2.0*sci.stats.norm.pdf(-X)-sci.stats.norm.pdf(X1)-sci.stats.norm.pdf(X2))+\
                    epsilon*(sci.stats.norm.cdf(X2)-sci.stats.norm.cdf(X1))
                    crit_dist = False
                    while crit_dist == False:
                        ind_max = np.argmax(EFF)
                        val_star = EFF[ind_max]
                        x_star = MC_sample[ind_max]
                        #x_star not in  DoE ?
                        if np.sqrt(np.sum((DoE-x_star)**2,axis=1)).min()>=np.sqrt(np.sum((DoE-x_star)**2,axis=1)).max()/10**3:
                            crit_dist = True
                        else:
                            crit_dist = False
                            EFF[ind_max] = -10**6
                    
                    print ("x min =",x_star)
                    print ("val min=", val_star)
                    y_star = performance_function(x_star)
                    new_DoE = np.zeros((DoE_size+1,DoE.shape[1]))
                    new_DoE[0:-1,:] = DoE
                    new_DoE[-1,:] = x_star
                    new_Y = np.zeros((DoE_size+1,1))
                    new_Y[0:-1,:] = Y
                    new_Y[-1,:] = y_star
                    DoE = new_DoE.copy()
                    Y = new_Y.copy()
                    DoE_size = DoE_size + 1
                    
      
        if Pf == 0. :
            # if no failure points : improvement of the GP or MC population enrichment 
            print('no failure point in MC')
            if n_iter<4 or n_iter >10: 
                print('GP improvement')
                epsilon = 2.0*np.sqrt(MSE)
                X = MC_sample_G[:,0]/np.sqrt(MSE)
                X1 = (-epsilon-MC_sample_G[:,0])/np.sqrt(MSE)
                X2 = (epsilon-MC_sample_G[:,0])/np.sqrt(MSE)
                EFF = MC_sample_G[:,0]*(2.0*sci.stats.norm.cdf(-X)-sci.stats.norm.cdf(X1)-sci.stats.norm.cdf(X2))-\
                np.sqrt(MSE)*(2.0*sci.stats.norm.pdf(-X)-sci.stats.norm.pdf(X1)-sci.stats.norm.pdf(X2))+\
                epsilon*(sci.stats.norm.cdf(X2)-sci.stats.norm.cdf(X1))
                crit_dist = False
                while crit_dist == False:
                    ind_max = np.argmax(EFF)
                    val_star = EFF[ind_max]
                    x_star = MC_sample[ind_max]
                    #x_star not in  DoE ?
                    if np.sqrt(np.sum((DoE-x_star)**2,axis=1)).min()>=np.sqrt(np.sum((DoE-x_star)**2,axis=1)).max()/10**3:
                        crit_dist = True
                    else:
                        crit_dist = False
                        EFF[ind_max] = -10**6                
                print ("x min =",x_star)
                print ("val min=", val_star)
                y_star = performance_function(x_star)
                new_DoE = np.zeros((DoE_size+1,DoE.shape[1]))
                new_DoE[0:-1,:] = DoE
                new_DoE[-1,:] = x_star
                new_Y = np.zeros((DoE_size+1,1))
                new_Y[0:-1,:] = Y
                new_Y[-1,:] = y_star
                DoE = new_DoE.copy()
                Y = new_Y.copy()
                DoE_size = DoE_size + 1
                
                
            else :     
                print( "NEW MC SAMPLE")
                #add  MC samples
                new_MC_sample = np.zeros((MC_sample.shape[0]+initial_MC_sample_size,MC_sample.shape[1]))
                new_MC_sample[0:MC_sample.shape[0],:] = MC_sample
                new_MC_sample[MC_sample.shape[0]:,:] = np.array(rv_distribution.getSample(initial_MC_sample_size))
                MC_sample = new_MC_sample.copy()
                MC_sample_size = MC_sample_size + initial_MC_sample_size
            
        n_iter = n_iter + 1
        MC_sample.dump(path+str('MC_sample.pk'))
        DoE.dump(path+str('DoE.pk'))
        Y.dump(path+str('Y.pk'))
     
    print('END Vb_AGP')
    return Pf,DoE,n_iter,gp_g,Y,MC_sample_size,np.sqrt(var_tot)/Pf,np.sqrt(var_tot_inf)/Pf,np.sqrt(var_tot_sup)/Pf




Code : analyse de sensibilité pour l'estimation de $V_{\tilde{X}}$ et $V_{\mathcal{G}_n}$ dans Vb-AGP

In [ ]:
def sensitivity_analysis(gp_g,distribution,MC_sample,pf,cov_max):
    
    Omega=np.array([[-4.,4.]*stochastic_dim]).reshape(stochastic_dim,2) # domain for the discretization for KL 
    eig, vect_star, nodes, weights, M =  gp_g.K_eig_noncond_randuniform(nb_vect=1000, Omega=Omega,normalize_Omega=False,n_start=2000,eps=1e-8)
    phi = gp_g.eval_phi_KL( weights, nodes, eig, vect_star, M, MC_sample) #evaluation of the eigenfunctions at the MC points
    # setup for GP trajectories simulation for V_G_E_X
    n_traj_max=15000 
    n_traj=2000
    
    pf_samples_list=list()
    phi_n = stats.norm()
    cond = False
    end_learning = False
    
    ### Computation of var_X_E_G
    MC_sample_size=MC_sample.shape[0]
    MC_sample_G,MSE=gp_g.predict(MC_sample,eval_MSE=True)
    U=-MC_sample_G[:,0]/np.sqrt(MSE)
    # computation of p_i
    p=phi_n.cdf(U.reshape((MC_sample_size,1)))
    var_X_E_G=np.var(p)/MC_sample_size
    # computation of var_X_E_G confidence interval 
    n=MC_sample_size
    p_mean=p.mean()
    v=var_X_E_G
    v_k=np.var((p-p_mean)**2)/(n*(n-1)**2)
    var_X_E_G_inf=v-3*np.sqrt(v_k)
    var_X_E_G_sup=v+3*np.sqrt(v_k)
    
    ### Computation of var_G_E_X
    cond =False
    pf_samples=np.array([0])
    while not(cond and pf_samples.shape[0] > 3000 or (pf_samples.shape[0] > n_traj_max) and pf_samples.shape[0] > 3000) :  
        # simu trajectoires
        xi_traj=np.random.normal(0,1,(M,n_traj))
        traj,phi=gp_g.eval_KL_expansion_trajectories_phi(weights=weights,nodes=nodes,eig=eig,vect=vect_star, M=M, x_new=MC_sample,xi_traj=xi_traj,phi=phi)
        # calcul des Pf par traj 
        indicatrice = np.zeros((MC_sample_size,n_traj))
        ind = traj.T <=0.0
        indicatrice[ind] = 1.0
        pf_mu= indicatrice.sum(axis=0)/MC_sample_size
        pf_samples=pf_mu
        pf_samples_list.append(pf_samples)
        pf_samples=np.array(pf_samples_list).flatten()
        pf_tot_mean=np.mean(pf_samples)

        var_G_E_X=((pf_samples-pf_tot_mean)**2).sum()/(pf_samples.shape[0]-1)  
        print('n_traj :',pf_samples.shape[0])
        # computation of var_G_E_X confidence interval 
        n=pf_samples.shape[0]
        v=var_G_E_X
        v_k=np.var((pf_samples-pf_tot_mean)**2)*n/(n-1)**2
        var_G_E_X_inf=v-3*np.sqrt(v_k)
        var_G_E_X_sup=v+3*np.sqrt(v_k)
        # verification that the intersection of the two confidence intervals is null 
        if not((var_G_E_X_inf < var_X_E_G_inf < var_G_E_X_sup) or (var_G_E_X_inf < var_X_E_G_sup < var_G_E_X_sup)):
            cond= True
            
    # verification of the 1st stopping criterion (COV_red)
    var_target=(cov_max*pf)**2
    if (var_G_E_X_sup+ var_X_E_G_sup < var_target):
        print('COV_red = ',np.sqrt(var_G_E_X+ var_X_E_G)/pf)
        end_learning=True      
     
    
    return var_X_E_G, var_G_E_X,end_learning,var_X_E_G_inf,var_X_E_G_sup,var_G_E_X_inf,var_G_E_X_sup



Code : Estimation de $V_{tot}$ et $\hat{P}_f^t$ par Bootstrap 

In [ ]:
def total_variance_bootstrap(gp_g,distribution,MC_sample_size,pf,cov_max,MC_sample):
    stochastic_dim=gp_g.X.shape[1]
    Omega=np.array([[-4.,4.]*stochastic_dim]).reshape(stochastic_dim,2)
    eig, vect_star, nodes, weights, M =  gp_g.K_eig_noncond_randuniform(nb_vect=1000, Omega=Omega,normalize_Omega=False,n_start=2000,eps=1e-8)
    phi = gp_g.eval_phi_KL( weights, nodes, eig, vect_star, M, MC_sample)
    # number of realizations of MC and GP 
    N_mc_i=500 
    N_mc_max=2000 
    pf_list=list()
    var_target=(cov_max*pf)**2
    var_tot_inf=var_target-1
    var_tot_sup=var_target+1
    N_mc=N_mc_i
    traj_ind=np.arange(MC_sample_size)
    while not( not(var_tot_inf<var_target<var_tot_sup) or N_mc>N_mc_max) :
        # simulation of n_traj on the MC population 
        n_traj=N_mc_i
        xi_traj=np.random.normal(0,1,(M,n_traj))
        traj_i,phi=gp_g.eval_KL_expansion_trajectories_phi(weights=weights,nodes=nodes,eig=eig,vect=vect_star, M=M, x_new=MC_sample,xi_traj=xi_traj,phi=phi)
        # Boostrap to simulate different MC population 
        for i in np.arange(N_mc_i):
            ind_boot=np.random.choice(traj_ind,size=MC_sample_size,replace=True)
            traj=traj_i.T[ind_boot,i]
            indicatrice = np.zeros((MC_sample_size,n_traj))
            ind = traj <=0.0
            indicatrice[ind] = 1.0
            indicatrice=indicatrice
            Pf = indicatrice.sum(axis=0)/MC_sample_size
            pf_list.append(Pf)
        
        pf=np.array(pf_list).flatten()
        pf_tot_mean=np.mean(pf)
        var_tot=((pf-pf_tot_mean)**2).sum()/(pf.shape[0]-1)     
        n=pf.shape[0]
        v=var_tot
        v_k=np.var((pf-pf_tot_mean)**2)*n/(n-1)**2
        var_tot_inf=v-3*np.sqrt(v_k)
        var_tot_sup=v+3*np.sqrt(v_k)
        N_mc=N_mc+N_mc_i 

    return var_tot,var_tot_inf,var_tot_sup,pf_tot_mean

Choix des paramètres de l'algorithme 

In [ ]:
###########  choix cas test + distribution et param algo 
fct_test= f_4b
distrib_test=distribution_4b
initial_doe_size=16
initial_MC_sample_size=5000
cov_max=0.1
iter_max = 20

In [ ]:
Pf,DoE,n_iter,gp_g, Y,MC_sample_size,cov_tot, cov_tot_inf,cov_tot_sup= Vb_AGP(fct_test,distrib_test,initial_MC_sample_size = initial_MC_sample_size,initial_doe_size = initial_doe_size,  iter_max = iter_max, cov_max=cov_max,corr='matern52')


In [ ]:
print('Pf = ',Pf)
print('COV total = ', cov_tot)
print('DoE size = ', int(DoE.shape[0]))
print('MC size = ',MC_sample_size)